# Library Imports

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
from torch import nn, optim
from torch.utils.data import Dataset
from torch.utils.data import DataLoader as DL
from torch.nn.utils import weight_norm as WN
import torch.nn.functional as F

import gc
import os
import cv2
from time import time

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

seed = 42

# Helper Functions

In [ ]:
def breaker():
    print("\n" + 50*"-" + "\n")

def head(x, no_of_ele=5):
    print(x[:no_of_ele])
    
def getImages(file_path=None, file_names=None, size=None):
    images = []
    for name in file_names:
        try:
            image = cv2.imread(file_path + name + ".jpg", cv2.IMREAD_GRAYSCALE).astype("float64")
        except AttributeError:
            print(file_path + name)
        if size:
            image = cv2.resize(image, dsize=(size, size), interpolation=cv2.INTER_LANCZOS4)
        cv2.normalize(src=image, dst=image, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX)
        images.append(image.reshape(1, size, size))
    return np.array(images)

# Data Handling

**Loading Image Data**

In [ ]:
start_time = time()

ss = pd.read_csv("../input/ranzcr-clip-catheter-line-classification/sample_submission.csv")

ts_img_names = ss["StudyInstanceUID"].values
ts_images = getImages("../input/ranzcr-clip-catheter-line-classification/test/", 
                      ts_img_names, 
                      size=144)

breaker()
print("Time Taken to read data : {:.2f} minutes".format((time() - start_time)/60))
breaker()

**Dataset Template**

In [ ]:
class Dataset(Dataset):
    def __init__(this, X=None, y=None, mode="train"):
        this.mode = mode
        this.X = X
        if mode == "train":
            this.y = y
            
    def __len__(this):
        return this.X.shape[0]
    
    def __getitem__(this, idx):
        if this.mode == "train":
            return torch.FloatTensor(this.X[idx]), torch.FloatTensor(this.y[idx])
        else:
            return torch.FloatTensor(this.X[idx])

# CNN Configuration and Setup

**Config**

In [ ]:
class CFG():
    tr_batch_size = 128 # Alos va_batch_size
    ts_batch_size = 128
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    in_channels = 1
    OL = 11
    
    def __init__(this, filter_sizes=[64, 128, 256, 512], HL=[4096, 4096], epochs=50, n_folds=5):
        this.filter_sizes = filter_sizes
        this.HL = HL
        this.epochs = epochs
        this.n_folds = n_folds

**Setup**

In [ ]:
class CNN(nn.Module):
    def __init__(this, in_channels=1, filter_sizes=None, HL=None, OL=None, use_DP=False, DP1=0.2, DP2=0.5):
        super(CNN, this).__init__()
        
        this.use_DP = use_DP
        
        this.DP1 = nn.Dropout(p=0.2)
        this.DP2 = nn.Dropout(p=0.5)
        
        this.MP_ = nn.MaxPool2d(kernel_size=2)
        
        this.CN1 = nn.Conv2d(in_channels=in_channels, out_channels=filter_sizes[0], kernel_size=3, stride=1, padding=1)
        this.BN1 = nn.BatchNorm2d(num_features=filter_sizes[0], eps=1e-5)
        
        this.CN2 = nn.Conv2d(in_channels=filter_sizes[0], out_channels=filter_sizes[1], kernel_size=3, stride=1, padding=1)
        this.BN2 = nn.BatchNorm2d(num_features=filter_sizes[1], eps=1e-5)
        
        this.CN3 = nn.Conv2d(in_channels=filter_sizes[1], out_channels=filter_sizes[2], kernel_size=3, stride=1, padding=1)
        this.BN3 = nn.BatchNorm2d(num_features=filter_sizes[2], eps=1e-5)
    
        this.CN4 = nn.Conv2d(in_channels=filter_sizes[2], out_channels=filter_sizes[3], kernel_size=3, stride=1, padding=1)
        this.BN4 = nn.BatchNorm2d(num_features=filter_sizes[3], eps=1e-5)
        
        this.CN5 = nn.Conv2d(in_channels=filter_sizes[3], out_channels=filter_sizes[3], kernel_size=3, stride=1, padding=1)
        this.BN5 = nn.BatchNorm2d(num_features=filter_sizes[3], eps=1e-5)
        
        this.CN6 = nn.Conv2d(in_channels=filter_sizes[3], out_channels=filter_sizes[3], kernel_size=3, stride=1, padding=1)
        this.BN6 = nn.BatchNorm2d(num_features=filter_sizes[3], eps=1e-5)
        
        this.FC1 = nn.Linear(in_features=filter_sizes[3]*2*2, out_features=HL[0])
        this.FC2 = nn.Linear(in_features=HL[0], out_features=HL[1])
        this.FC3 = nn.Linear(in_features=HL[1], out_features=OL)
        
    def getOptimizer(this, A_S=True, lr=1e-3, wd=0):
        if A_S:
            return optim.Adam(this.parameters(), lr=lr, weight_decay=wd)
        else:
            return optim.SGD(this.parameters(), lr=lr, momentum=0.9, weight_decay=wd)

    def getStepLR(this, optimizer=None, step_size=5, gamma=0.1):
        return optim.lr_scheduler.StepLR(optimizer=optimizer, step_size=step_size, gamma=gamma)

    def getMultiStepLR(this, optimizer=None, milestones=None, gamma=0.1):
        return optim.lr_scheduler.MultiStepLR(optimizer=optimizer, milestones=milestones, gamma=gamma)
        
    def getPlateauLR(this, optimizer=None, patience=5, eps=1e-6):
        return optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer, patience=patience, eps=eps, verbose=True)
    
    def forward(this, x):
        if not this.use_DP:
            x = F.relu(this.MP_(this.BN1(this.CN1(x))))
            x = F.relu(this.MP_(this.BN2(this.CN2(x))))
            x = F.relu(this.MP_(this.BN3(this.CN3(x))))
            x = F.relu(this.MP_(this.BN4(this.CN4(x))))
            x = F.relu(this.MP_(this.BN5(this.CN5(x))))
            x = F.relu(this.MP_(this.BN6(this.CN6(x))))
            
            x = x.view(x.shape[0], -1)
            
            x = F.relu(this.FC1(x))
            x = F.relu(this.FC2(x))
            x = this.FC3(x)
            
            return x
        else:
            x = F.relu(this.MP_(this.BN1(this.CN1(x))))
            x = F.relu(this.MP_(this.BN2(this.CN2(x))))
            x = F.relu(this.MP_(this.BN3(this.CN3(x))))
            x = F.relu(this.MP_(this.BN4(this.CN4(x))))
            x = F.relu(this.MP_(this.BN5(this.CN5(x))))
            x = F.relu(this.MP_(this.BN6(this.CN6(x))))
            
            x = x.view(x.shape[0], -1)
            
            x = F.relu(this.DP2(this.FC1(x)))
            x = F.relu(this.DP2(this.FC2(x)))
            x = this.FC3(x)
            
            return x

**Predict Function**

In [ ]:
def predict_(model=None, dataloader=None, device=None, path=None):
    if path:
        model.load_state_dict(torch.load(path))
    else:
        pass
    model.to(device)
    model.eval()

    y_pred = torch.zeros(1, 11).to(device)

    for X in dataloader:
        X = X.to(device)
        with torch.no_grad():
            Pred = torch.sigmoid(model(X))
        y_pred = torch.cat((y_pred, Pred), dim=0)
    
    return y_pred[1:].detach().cpu().numpy()

# Submission 

In [ ]:
cfg = CFG(filter_sizes=[64, 128, 256, 512], HL=[4096, 4096], epochs=50, n_folds=5)

ts_data_setup = Dataset(ts_images, None, "test")
ts_data = DL(ts_data_setup, batch_size=cfg.ts_batch_size, shuffle=False)

model = CNN(in_channels=cfg.in_channels, filter_sizes=cfg.filter_sizes, HL=cfg.HL, OL=cfg.OL)

y_pred_e19 = predict_(model=model, dataloader=ts_data, device=cfg.device, path="../input/rccl-1x144-f-train/Epoch_19.pt")
y_pred_e20 = predict_(model=model, dataloader=ts_data, device=cfg.device, path="../input/rccl-1x144-f-train/Epoch_20.pt")
y_pred_e23 = predict_(model=model, dataloader=ts_data, device=cfg.device, path="../input/rccl-1x144-f-train/Epoch_23.pt")
y_pred_e25 = predict_(model=model, dataloader=ts_data, device=cfg.device, path="../input/rccl-1x144-f-train/Epoch_25.pt")

y_pred = (y_pred_e19 + y_pred_e20 + y_pred_e23 + y_pred_e25)/4
y_pred = np.clip(y_pred, 1e-15, 1-1e-15)

ss.iloc[:, 1:] = y_pred
ss.to_csv("./submission.csv", index=False)
ss.head(5)